# Subspace Model

This notebook illustrate how to use a Bayesian Normal density model with the [beer framework](https://github.com/beer-asr/beer). The Normal distribution is a fairly basic models but it is used extenslively in other model as a basic building block.

In [1]:
# Add "beer" to the PYTHONPATH
import sys
sys.path.insert(0, '../')

import beer
import numpy as np
import torch

# For plotting.
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, gridplot
from bokeh.models import Arrow, OpenHead, NormalHead, VeeHead
output_notebook()

# Convenience functions for plotting.
import plotting

%load_ext autoreload
%autoreload 2

Loading BokehJS ...

## Data

Generate some normally distributed data:

In [50]:
mean = np.array([-10, 10]) 
cov = .5 * np.array([[3, -1.5], [-1.5, 2]])
#cov = np.eye(2) 
data = np.random.multivariate_normal(mean, cov, size=300)

x_range = (mean[0] - 5, mean[0] + 5)
y_range = (mean[1] - 5, mean[1] + 5)

fig = figure(
    title='Data',
    width=400,
    height=400,
    x_range=x_range,
    y_range=y_range
)
fig.circle(data[:, 0], data[:, 1])
plotting.plot_normal(fig, mean, cov, alpha=.5, color='black')

show(fig)

## Model Creation

We create two types of Normal distribution: one diagonal covariance matrix and another one with full covariance matrix.

In [51]:
model = beer.PPCA.create(torch.zeros(2).double(), 1, torch.randn(1, 2).double())
#model = beer.NormalDiagonalCovariance.create(torch.zeros(2).double(), torch.ones(2).double())

## Variational Bayes Training 

In [52]:
epochs = 100
lrate = 1
X = torch.from_numpy(data).double()
elbo_fn = beer.EvidenceLowerBound(len(X))
params = model.parameters
optimizer = beer.BayesianModelOptimizer(params, lrate)
    
elbos = []
klds = []
for epoch in range(epochs):
    optimizer.zero_grad()
    elbo = elbo_fn(model, X)
    elbo.natural_backward()
    optimizer.step()
    elbos.append(round(float(elbo) / len(X), 5))
    klds.append(round(float(elbo._kl_div) / len(X), 2))
fig = figure(width=400, height=400)
fig.line(range(len(elbos)), elbos)
#fig.line(range(len(elbos)), klds, color='green')
show(fig)

In [53]:
model.precision

tensor(4.0323, dtype=torch.float64)

In [54]:
mean = model.mean.numpy()
evectors = model.subspace.numpy().T
print('bases:', evectors, evectors.shape)

fig1 = figure(
    title='PPCA',
    x_range=x_range,
    y_range=y_range,
    width=400,
    height=400
)

fig1.circle(data[:, 0], data[:, 1], alpha=.3)

# Plot subspace.
alpha = np.linalg.norm(evectors[:, 0])
alpha = 1
fig1.add_layout(
    Arrow(
        end=NormalHead(size=10, line_alpha=alpha, fill_alpha=alpha),
        line_width=2,
        line_alpha=alpha,
        x_start=mean[0], 
        y_start=mean[1], 
        x_end=evectors[0, 0] + mean[0], 
        y_end=evectors[1, 0] + mean[1])
)

X = data
C = cov
evals, evectors = np.linalg.eigh(C)
print('evectors')
print(evectors)

fig2 = figure(
    title='PCA',
    x_range=x_range,
    y_range=y_range,
    width=400,
    height=400
)

fig2.circle(data[0], data[1], alpha=.3)

# Plot subspace.
alpha = np.linalg.norm(evectors[:, 1])
fig2.add_layout(
    Arrow(
        end=NormalHead(size=10, line_alpha=alpha, fill_alpha=alpha),
        line_width=2,
        line_alpha=alpha,
        x_start=mean[0], 
        y_start=mean[1], 
        x_end=evectors[0, 0] + mean[0], 
        y_end=evectors[1, 0] + mean[1])
)
alpha = np.linalg.norm(evectors[:, 1])
fig2.add_layout(
    Arrow(
        end=NormalHead(size=10, line_alpha=alpha, fill_alpha=alpha),
        line_width=2,
        line_alpha=alpha,
        x_start=mean[0], y_start=mean[1], 
        x_end=evectors[0, 1] + mean[0], 
        y_end=evectors[1, 1] + mean[1])
)

grid = gridplot([[fig1, fig2]])
show(grid)

bases: [[-1.04449414]
 [ 0.77593919]] (2, 1)
evectors
[[-0.58471028 -0.81124219]
 [-0.81124219  0.58471028]]
